# 一个完整的业务解决方案

## 现在，我们将把第一天的项目提升到一个新的水平

### 业务挑战：

创建一个产品，为公司制作宣传册，供潜在客户、投资者和潜在员工使用。

我们将获得一个公司名称及其主网站。

请参阅本笔记本末尾的真实世界业务应用示例。


In [1]:
# 导入库
# 如果这些导入失败，请检查您是否在命令提示符中带有 (llms) 的“已激活”环境中运行

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# 初始化和常量

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API 密钥目前看起来没问题")
else:
    print("您的 API 密钥可能有问题？请查看故障排除笔记本！")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API 密钥目前看起来没问题


In [3]:
# 一个表示网页的类

# 抓取某些网站时需要使用正确的请求头：
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    一个工具类，用于表示我们抓取的网站，现在包含链接
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "未找到标题"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"网页标题：\n{self.title}\n网页内容：\n{self.text}\n\n"

In [4]:
ed = Website("https://iloveaws.cn")
ed.links

['https://www.iloveaws.cn',
 'https://www.iloveaws.cn/all-courses',
 'https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81',
 'https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/01-%e5%bc%80%e5%a7%8b%e8%af%be%e7%a8%8b',
 'https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/02-%e8%ba%ab%e4%bb%bd%e4%b8%8e%e8%81%94%e5%90%88%e8%ba%ab%e4%bb%bd%e9%aa%8c%e8%af%81',
 'https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/03-%e8%ae%a1%e7%ae%97%e4%b8%8e%e8%b4%9f%e8%bd%bd%e5%9d%87%e8%a1%a1',
 'https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/04-%e5%ad%98%e5%82%a8',
 'https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/05-%e6%95%b0%e6%8d%ae%e5%ba%93',
 'https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/06-%e7%bc%93%e5%ad%98',
 'https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/07-vpc',
 'https://

## 第一步：让 GPT-4o-mini 找出哪些链接是相关的

### 调用 gpt-4o-mini 来读取网页上的链接，并以结构化的 JSON 格式响应。
它应该判断哪些链接是相关的，并将相对链接（如“/about”）替换为“https://company.com/about”。
我们将使用“单样本提示”（one shot prompting），即在提示中提供一个它应该如何响应的示例。

这是一个 LLM 的绝佳用例，因为它需要细致的理解。想象一下，如果没有 LLM，试图通过解析和分析网页来编写代码——这将非常困难！

旁注：有一种更高级的技术叫做“结构化输出”，我们要求模型根据规范进行响应。我们将在第 8 周的自主 Agentic AI 项目中介绍这项技术。

In [5]:
link_system_prompt = "您将收到一个在网页上找到的链接列表。\
您需要判断哪些链接最适合包含在公司宣传册中， \
例如指向“关于我们”页面、公司页面或“职业/工作”页面的链接。\n"
link_system_prompt += "您应该像此示例一样以 JSON 格式响应："
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [9]:
print(link_system_prompt)

您将收到一个在网页上找到的链接列表。您需要判断哪些链接最适合包含在公司宣传册中， 例如指向“关于我们”页面、公司页面或“职业/工作”页面的链接。
您应该像此示例一样以 JSON 格式响应：
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [10]:
def get_links_user_prompt(website):
    user_prompt = f"这是网站 {website.url} 上的链接列表 - "
    user_prompt += "请判断哪些是与公司宣传册相关的网页链接，并以 JSON 格式返回完整的 https URL。\
不要包含服务条款、隐私政策、电子邮件链接。\n"
    user_prompt += "链接（其中一些可能是相对链接）：\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [11]:
print(get_links_user_prompt(ed))

这是网站 https://iloveaws.cn 上的链接列表 - 请判断哪些是与公司宣传册相关的网页链接，并以 JSON 格式返回完整的 https URL。不要包含服务条款、隐私政策、电子邮件链接。
链接（其中一些可能是相对链接）：
https://www.iloveaws.cn
https://www.iloveaws.cn/all-courses
https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81
https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/01-%e5%bc%80%e5%a7%8b%e8%af%be%e7%a8%8b
https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/02-%e8%ba%ab%e4%bb%bd%e4%b8%8e%e8%81%94%e5%90%88%e8%ba%ab%e4%bb%bd%e9%aa%8c%e8%af%81
https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/03-%e8%ae%a1%e7%ae%97%e4%b8%8e%e8%b4%9f%e8%bd%bd%e5%9d%87%e8%a1%a1
https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/04-%e5%ad%98%e5%82%a8
https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/05-%e6%95%b0%e6%8d%ae%e5%ba%93
https://www.iloveaws.cn/category/%e9%80%9a%e8%bf%87aws-sap%e8%ae%a4%e8%af%81/06-%e7%bc%93%e5%ad%98
https://www.ilo

In [12]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [13]:
# Anthropic 的网站变得更难抓取，所以我改用 https://www.volcengine.com..

huggingface = Website("https://www.volcengine.com")
huggingface.links

['//www.volcengine.com',
 'https://www.volcengine.com/docs',
 'https://www.volcengine.com/beian',
 'https://console.volcengine.com',
 '//console.volcengine.com/auth/login?redirectURI=%2F%2Fwww.volcengine.com',
 '//console.volcengine.com/auth/signup?redirectURI=%2F%2Fwww.volcengine.com',
 'https://www.volcengine.com/event/force-2506',
 'https://www.volcengine.com/live/event/force-2506',
 'https://exp.volcengine.com',
 'https://exp.volcengine.com',
 'https://exp.volcengine.com',
 '/activity/new',
 '/activity/new',
 'https://console.volcengine.com/ark/region:ark+cn-beijing/model?vendor=Bytedance&view=LIST_VIEW',
 'https://exp.volcengine.com/ark?model=doubao-seed-1-6-250615',
 'https://console.volcengine.com/ark/region:ark+cn-beijing/experience/vision',
 'https://exp.volcengine.com/ark?model=doubao-1-5-thinking-vision-pro-250428',
 'https://console.volcengine.com/ark/region:ark+cn-beijing/experience/voice?type=RealTime',
 'https://console.volcengine.com/ark/region:ark+cn-beijing/experience

In [14]:
get_links("https://www.volcengine.com")

{'links': [{'type': 'company page', 'url': 'https://www.volcengine.com'},
  {'type': 'about page', 'url': 'https://www.volcengine.com/docs'},
  {'type': 'careers page', 'url': 'https://developer.volcengine.com/'},
  {'type': 'partner page', 'url': 'https://partner.volcengine.com/'},
  {'type': 'contact page',
   'url': 'https://www.volcengine.com/contact/doubao?t=豆包大模型'}]}

## 第二步：制作宣传册！

将所有细节整合到另一个发送给 GPT-4o 的提示中

In [15]:
def get_all_details(url):
    result = "着陆页：\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("找到的链接：", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [16]:
print(get_all_details("https://www.volcengine.com"))

找到的链接： {'links': [{'type': 'company homepage', 'url': 'https://www.volcengine.com'}, {'type': 'about page', 'url': 'https://www.volcengine.com/trust/home'}, {'type': 'careers page', 'url': 'https://developer.volcengine.com'}, {'type': 'contact page', 'url': 'https://www.volcengine.com/contact/doubao?t=豆包大模型'}, {'type': 'solutions page', 'url': 'https://www.volcengine.com/solutions/auto-AI-omnichannel-marketing'}, {'type': 'developer page', 'url': 'https://developer.volcengine.com/'}]}
着陆页：
网页标题：
火山引擎-云上增长新动力
网页内容：
最新活动
大模型
产品
解决方案
定价
生态与合作
支持与服务
开发者
了解我们
文档
备案
控制台
登录
立即注册
火山引擎-云上增长新动力
查看详情
观看回放
原动力大会
豆包大模型
市场份额
AI云原生
AI生态
即刻开启 AI 体验中心，体验最前沿能力
豆包大模型 1.6 全新上线
AI 体验中心
即刻开启
AI 体验中心
体验最前沿能力
豆包大模型 1.6 全新上线
极速体验实时多模态对话式AI
豆包大模型 1.6 全新上线
ECS 热门机型首年29元起
新用户特惠专场
促销活动
ECS 热门机型首年29元起
扣子限时9.9元/月
新用户特惠专场
新用户特惠专场
豆包大模型 · 更强模型、更低价格、更易落地
更强模型、更低价格、更易落地
豆包大模型
更强模型、更低价格、更易落地
查看全部模型
豆包大模型
1.6
HOT
更强的推理能力，多模态理解能力，GUI操作能力和前端页面编程能力
豆包 ·
视频生成
模型
NEW
模型支持将用户的文本、图像生成高质量的视频，能够生成具备丰富的细节层次的影视即视频
豆包 ·
视觉理解
模型
NEW
对

In [31]:
system_prompt = "你是一个助手，负责分析公司网站上几个相关页面的内容，\
并为潜在客户、投资者和应聘者创建一份关于公司的简短宣传册。请以 markdown 格式回应。\
如果信息可用，请包含公司文化、客户和职业/工作等细节。"

# 或者取消下面几行的注释以获得更幽默的宣传册 - 这展示了融入“语气”是多么容易：

#system_prompt = "你是一个助手，负责分析公司网站上几个相关页面的内容，\
#并为潜在客户、投资者和应聘者创建一份关于公司的简短、幽默、有趣的宣传册。请以 markdown 格式回应。\
#如果信息可用，请包含公司文化、客户和职业/工作等细节。"

In [23]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"你正在查看的公司是：{company_name}\n"
    user_prompt += f"这是其着陆页和其他相关页面的内容；请使用这些信息以 markdown 格式为该公司制作一份简短的宣传册。\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # 如果超过 5000 个字符则截断
    return user_prompt

In [25]:
get_brochure_user_prompt("豆包大模型", "https://www.volcengine.com")

找到的链接： {'links': [{'type': 'company page', 'url': 'https://www.volcengine.com'}, {'type': 'about page', 'url': 'https://www.volcengine.com/docs'}, {'type': 'careers page', 'url': 'https://developer.volcengine.com/'}, {'type': 'solutions page', 'url': 'https://www.volcengine.com/solutions/auto-AI-omnichannel-marketing'}, {'type': 'developer page', 'url': 'https://developer.volcengine.com/articles/7491601165575012406'}, {'type': 'partner page', 'url': 'https://partner.volcengine.com/'}, {'type': 'contact page', 'url': 'https://www.volcengine.com/contact/doubao?t=豆包大模型'}]}


'你正在查看的公司是：豆包大模型\n这是其着陆页和其他相关页面的内容；请使用这些信息以 markdown 格式为该公司制作一份简短的宣传册。\n着陆页：\n网页标题：\n火山引擎-云上增长新动力\n网页内容：\n最新活动\n大模型\n产品\n解决方案\n定价\n生态与合作\n支持与服务\n开发者\n了解我们\n文档\n备案\n控制台\n登录\n立即注册\n火山引擎-云上增长新动力\n查看详情\n观看回放\n原动力大会\n豆包大模型\n市场份额\nAI云原生\nAI生态\n即刻开启 AI 体验中心，体验最前沿能力\n豆包大模型 1.6 全新上线\nAI 体验中心\n即刻开启\nAI 体验中心\n体验最前沿能力\n豆包大模型 1.6 全新上线\n极速体验实时多模态对话式AI\n豆包大模型 1.6 全新上线\nECS 热门机型首年29元起\n新用户特惠专场\n促销活动\nECS 热门机型首年29元起\n扣子限时9.9元/月\n新用户特惠专场\n新用户特惠专场\n豆包大模型 · 更强模型、更低价格、更易落地\n更强模型、更低价格、更易落地\n豆包大模型\n更强模型、更低价格、更易落地\n查看全部模型\n豆包大模型\n1.6\nHOT\n更强的推理能力，多模态理解能力，GUI操作能力和前端页面编程能力\n豆包 ·\n视频生成\n模型\nNEW\n模型支持将用户的文本、图像生成高质量的视频，能够生成具备丰富的细节层次的影视即视频\n豆包 ·\n视觉理解\n模型\nNEW\n对视觉内容有更强的识别能力，更强理解和推理能力，以及更细腻的视觉描述能力\n豆包 ·\n实时语音\n模型\n真正的端到端语音模型，通过自然语言进行多种高级指令控制，并能唱会演具备超拟人交互能力\n豆包 ·\n文生图\n模型\n快速生成精美写真，支持50余种风格变换，并对图片实现扩图、重绘、涂抹等创意延展\n豆包 ·\n声音复刻\n模型\n5秒即可实现声音1:1克隆，对音色相似度和自然度进行高度还原，支持声音的跨语种迁移\n查看全部模型\n体验豆包大模型 1.6\n1\n/\n2\n立\n即\n体\n验\n体验完整能力，请访问\nAI体验中心\n体验豆包大模型 1.6\n立\n即\n体\n验\n体验完整能力，请访问\nAI体验中心\n体验实时对话式 AI\n1\n/\n2\n立\n即\n体\n验\n体验完整能力

In [26]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [27]:
create_brochure("豆包大模型", "https://www.volcengine.com")

找到的链接： {'links': [{'type': 'about page', 'url': 'https://www.volcengine.com'}, {'type': 'careers page', 'url': 'https://developer.volcengine.com/'}, {'type': 'corporate page', 'url': 'https://partner.volcengine.com/'}, {'type': 'solutions page', 'url': 'https://www.volcengine.com/solutions/auto-AI-omnichannel-marketing'}, {'type': 'contact page', 'url': 'https://www.volcengine.com/contact/doubao?t=豆包大模型'}]}


# 豆包大模型公司宣传册

## 公司介绍
**豆包大模型** 是火山引擎旗下的一项重要AI技术，致力于通过云计算和大数据，为企业提供强大的人工智能解决方案。我们的目标是为客户创造更高效、更智能的业务环境，通过领先的技术与服务推动商业增长。

## 产品与服务
### 豆包大模型 1.6
- **多模态对话式AI**：具备实时响应和丰富理解能力，能够处理文本、图像、音频等多种输入，将用户体验提升到新高度。
- **视频生成模型**：支持将用户的文本和图像转化为高质量视频，具有丰富的细节描述能力。
- **视觉理解模型**：强大的视觉内容识别和推理能力，能够生成更细致的描述。
- **实时语音模型**：真正的端到端语音处理能力，通过自然语言进行高级指令控制。
- **声音复刻模型**：快速实现声音的1:1克隆，支持跨语种迁移。

### 平台与合作
- **火山方舟**：一站式大模型开发平台，提供企业用户50万Tokens免费额度，支持业务快速落地。
- **API构建应用**：提供全面的模型精调、推理和评测功能，保障企业级AI应用的安全与稳定。
- **助手开发平台扣子**：支持多种开发模式和框架，便捷的AI Agent开发环境。

## 公司文化
在豆包大模型，我们倡导创新、合作与技术导向的文化。我们鼓励员工不断学习、分享知识，并在实践中探索新的解决方案，让团队共同成长。

## 客户群体
我们的解决方案被广泛应用于多个行业，包括但不限于：
- **汽车行业**：运用AI提升用户体验和满意度。
- **金融 sector**：提供智能化风险控制和客户管理工具。
- **教育和科研**：助力个性化学习和高效的数据分析。

## 职业机会
豆包大模型欢迎热爱科技与创新的人才加入我们。无论您是技术专家、产品经理还是市场营销人员，我们都期待能够与您一起共创未来。

## 联系我们
欲了解更多有关豆包大模型的信息，请访问我们的网站或联系我们的支持团队。我们期待与您携手合作，共同迎接AI时代的挑战与机遇！

[访问我们的网站](#)  
[立即注册体验](#)  

### 安全与合规
我们致力于提供安全、透明和可信赖的服务，获得了多个权威认证，包括ISO 27001和ISO 27701等，确保客户的数据安全及隐私保护。

---

在豆包大模型，我们每一步都在追求卓越，为客户提供最优质的服务与解决方案。期待与您同行，共同创造未来的可能性！

## 最后 - 一个小改进

通过一个小小的调整，我们可以让结果从 OpenAI 以流式传输回来，
并带有熟悉的打字机动画效果

In [28]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [30]:
stream_brochure("豆包大模型", "https://www.volcengine.com")

找到的链接： {'links': [{'type': 'company page', 'url': 'https://www.volcengine.com'}, {'type': 'about page', 'url': 'https://www.volcengine.com/docs'}, {'type': 'careers page', 'url': 'https://developer.volcengine.com/'}, {'type': 'product page', 'url': 'https://www.volcengine.com/product/list'}, {'type': 'solutions page', 'url': 'https://www.volcengine.com/solutions/auto-AI-omnichannel-marketing'}]}


# 豆包大模型宣传册

## 公司简介
豆包大模型隶属于火山引擎，致力于为用户提供最前沿的人工智能解决方案。我们专注于大模型开发，通过技术的不断创新与优化，赋能各行各业，让企业和个人能够高效利用AI技术。

## 公司文化
我们秉承“云上增长新动力”的理念，通过开放、合作的姿态，携手各界伙伴致力于打造高效、安全、透明的人工智能生态系统。我们的团队鼓励创新，强调用户体验，力求每个解决方案都能为客户创造真正的价值。

## 主要产品与解决方案
### 豆包大模型 1.6
- **推理能力更强**：多模态理解与更高的交互能力，支持文本、图像和语音输入。
- **实时语音模型**：具备自然语言指令控制与超拟人交互能力。
- **视频生成与视觉理解模型**：能够将文本和图像转化为高质量视频，具备细腻的视觉描述能力。

### 开发平台
- **火山方舟**：一站式大模型开发平台，为企业和开发者提供丰富的模型和API服务。
- **扣子开发平台**：提供多种开发模式与框架，简化AI Agent的开发与部署。

### 安全智能体
- 提供24小时安全监控与威胁研判服务，确保运维效率和安全性。

## 用户与案例
我们的解决方案服务于多个行业，包括汽车、金融、教育和医疗等。与众多知名企业合作，例如梅赛德斯-奔驰，我们帮助其提升智能交互体验。

## 投资者关心的信息
豆包大模型目前正与多家企业合作，致力于通过不断创新来提升市场份额。我们的平台提供安全合规性的诚信保障，包括ISO 27001及CMMI认证。

## 应聘者信息
我们欢迎充满激情、具备创新精神的技术人才加入我们的团队。如果你想在快速发展的AI行业中发挥自己的才华，豆包大模型是你理想的选择。我们提供多样的职业发展机会和良好的工作环境。

## 联系我们
- **官方网站**：访问我们的 [AI体验中心](#) 以了解更多信息。
- **支持与服务**：有任何问题，请联系我们的客服团队，我们承诺为您提供最佳的支持与服务。

## 结束语
豆包大模型将持续引领人工智能的创新与发展，不断为用户带来更强大的智能体验。我们期待与您携手共创未来！

In [32]:
# 尝试在为 豆包大模型 制作宣传册时，将系统提示更改为幽默版本：

stream_brochure("豆包大模型", "https://www.volcengine.com")

找到的链接： {'links': [{'type': 'company page', 'url': 'https://www.volcengine.com'}, {'type': 'about page', 'url': 'https://www.volcengine.com/docs'}, {'type': 'careers page', 'url': 'https://developer.volcengine.com/'}, {'type': 'contact page', 'url': 'https://www.volcengine.com/contact/doubao?t=豆包大模型'}, {'type': 'solutions page', 'url': 'https://www.volcengine.com/solutions/auto-AI-omnichannel-marketing'}]}


# 豆包大模型简短宣传册

---

## 🌋 欢迎来到豆包大模型的火山引擎!
### 云上增长新动力

你是否也希望拥有一个能给你带来高效、便捷，甚至偶尔“调皮”的 AI 助手？那么你来对地方了！

---

## 🤖 我们的明星产品：豆包大模型 1.6

**更强模型、更低价格、更易落地！**
- **多模态对话式AI**：和豆包大模型聊天，简直比和你隔壁邻居聊八卦还有趣。
- **视觉理解**：想让 AI 理解你的狗狗有多可爱？没问题，豆包大模型能帮你做到！
- **视频生成**：用物品生成一部好莱坞大片？没错，今天就让豆包大模型帮你完成！

---

## 🚀 立即体验我们前沿的 AI 能力

只需访问 **[AI 体验中心](#)**，就能体会到豆包大模型的超能力！快来体验一把！

---

## 🏢 公司文化：开放、协作、创新

在豆包大模型，我们坚信团队的力量。每一位成员都能在开放的氛围中畅所欲言，共同开发出最优秀的产品。快来加入我们，带着幽默的灵魂一起探索更广阔的可能性！

---

## 👥 我们的客户群体

无论你是汽车行业的创新者、教育界的先锋，还是在科技前沿奋战的勇士，豆包大模型都能为你定制化服务——帮助你在竞争中保持领先。

---

## 🎉 新用户特惠

首次注册立享 **ECS 热门机型首年仅需 29 元起**，还有更多超值优惠等你来领哦！快来让你的AI梦起航吧！

---

## 📞 加入我们！

不论是潜在客户、投资者，或者是追求职业发展的新伙伴，我们都热忱欢迎！成为我们的一员，共同在 AI 的世界中发现乐趣和可能。

---

**豆包大模型，为你开启全新智能生活的无限可能！**

---

> (注：实际价格和服务请以官方网站为准）

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">业务应用</h2>
            <span style="color:#181;">在本练习中，我们扩展了第一天的代码，以进行多次 LLM 调用并生成文档。

这可能是 Agentic AI 设计模式的第一个例子，因为我们组合了对 LLM 的多次调用。这将在第 2 周更多地出现，然后我们将在第 8 周构建完全自主的 Agent 解决方案时，再次大规模地回归 Agentic AI。

以这种方式生成内容是最常见的用例之一。与摘要一样，这可以应用于任何业务垂直领域。撰写营销内容，根据规范生成产品教程，创建个性化电子邮件内容等等。探索如何将内容生成应用于您的业务，并尝试为自己制作一个概念验证原型。</span>
        </td>
    </tr>
</table>